## Imports

In [2]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

In [3]:
!echo $CUDA_VISIBLE_DEVICES

0,1


In [4]:
DEVICE = torch.device('cuda')
DEVICE

device(type='cuda')

## Run in example

In [10]:
%run -n ../eval_classification_grad_cam.py

In [ ]:
run_name = '0715_232418_cxr14_densenet-121_lr1e-06_os-max4_aug-0_Pneumonia'
debug_run = False

In [11]:
run_evaluation(run_name,
               debug=debug_run,
               device='cuda',
               max_samples=None,
               batch_size=10,
               thresh=0.5,
               image_size=None,
               quiet=False,
               multiple_gpu=False,
              )

NameError: name 'run_name' is not defined